In [3]:
from datetime import datetime, time, UTC

def main():
    dict_value = '2024-05-09T12:22:13.830246+01:00'

    try:
        date = datetime.fromisoformat(dict_value)
    except Exception:
        return "Key at could not be converted to datetime"

    if date > datetime.now(UTC):
        return "Key at is set in the future"

    return date.time().replace(microsecond=0)

print(main())

12:22:13


# Museum Analysis

This notebook connects to the museum database and conducts simple analysis of the data.

## Imports

In [71]:
from pipeline import get_db_connection

## Setup

In [72]:
conn = get_db_connection()
cur = conn.cursor()

## Questions

### What exhibition is most frequently visited?

In [73]:
cur.execute("""SELECT exhibition_name, COUNT(*) AS visit_count
            FROM review
            JOIN exhibition USING(exhibition_id)
            GROUP BY exhibition_name
            ORDER BY visit_count DESC""")

print(cur.fetchone())

('Our Polluted World', 481)


### What hour of the day has the most ratings?

In [74]:
cur.execute("""SELECT EXTRACT(HOUR FROM created_at)::INT AS hour_of_day, 
            COUNT(*) AS rating_count
            FROM review
            GROUP BY hour_of_day
            ORDER BY rating_count DESC""")

print(cur.fetchone())

(10, 232)


### What is the average rating for each exhibition?

In [75]:
cur.execute("""SELECT exhibition_name,
            AVG(rating_id)::FLOAT AS average_rating FROM review
            JOIN exhibition USING(exhibition_id)
            GROUP BY exhibition_name
            ORDER BY average_rating DESC""")

print(cur.fetchall())

[('Cetacean Sensations', 2.830508474576271), ('Adaptation', 1.9294117647058824), ('Measureless to Man', 1.9169435215946844), ('The Crenshaw Collection', 1.425974025974026), ('Our Polluted World', 1.2162162162162162)]


### What proportion of all 4+ ratings are given to exhibition 4?

In [76]:
cur.execute("""-- Total count of all 4+ ratings
            WITH FourPlusRatings AS (
                SELECT COUNT(*) AS total_count
                FROM review
                WHERE rating_id >= 3  -- Assuming rating_id 3 and above correspond to 4+ ratings
            ),

            -- Count of 4+ ratings for exhibition 4
            ExhibitionFourPlusRatings AS (
                SELECT COUNT(*) AS exhibition_count
                FROM review
                WHERE exhibition_id = 4  -- Assuming exhibition_id 4 corresponds to exhibition 4
                AND rating_id >= 3  -- Assuming rating_id 3 and above correspond to 4+ ratings
            )

            -- Calculate the proportion
            SELECT exhibition_count::FLOAT/total_count
            FROM FourPlusRatings, ExhibitionFourPlusRatings;
            """)

print(cur.fetchone())

(0.091324200913242,)


### Are positive ratings more frequent before or after 1pm?

In [77]:
cur.execute("""WITH RatingsBefore1PM AS (
                SELECT COUNT(*) AS count_before
                FROM review
                WHERE EXTRACT(HOUR FROM created_at) < 13  -- Before 1 PM
                AND rating_id >= 3  -- Positive ratings
            ),
            RatingsAfter1PM AS (
                SELECT COUNT(*) AS count_after
                FROM review
                WHERE EXTRACT(HOUR FROM created_at) >= 13  -- After 1 PM
                AND rating_id >= 3  -- Positive ratings
            )
            SELECT 
                CASE 
                    WHEN count_before > count_after THEN 'Positive ratings more frequent before 1pm'
                    WHEN count_before < count_after THEN 'Positive ratings more frequent after 1pm'
                    ELSE 'Positive ratings the same before and after 1pm'
                END AS rating_comparison
            FROM RatingsBefore1PM, RatingsAfter1PM;""")

print(cur.fetchone())

('Positive ratings more frequent after 1pm',)


### How many ratings each hour are above the average rating for exhibition 4?

In [78]:
cur.execute("""WITH ExhibitionFourAvg AS (
                SELECT AVG(rating_id) AS avg_rating
                FROM review
                WHERE exhibition_id = 4
            ),
            RatingsAboveAvg AS (
                SELECT EXTRACT(HOUR FROM created_at) AS hour_of_day, COUNT(*) AS above_avg_count
                FROM review
                JOIN exhibition USING(exhibition_id)
                CROSS JOIN ExhibitionFourAvg
                WHERE rating_id > (SELECT avg_rating FROM ExhibitionFourAvg)
                AND exhibition_id = 4
                GROUP BY hour_of_day
            )
            SELECT hour_of_day::INT, COALESCE(above_avg_count, 0) AS ratings_above_avg
            FROM generate_series(0, 23) AS hours(hour_of_day)
            LEFT JOIN RatingsAboveAvg USING (hour_of_day)
            ORDER BY hour_of_day;""")

print(cur.fetchall())

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 15), (9, 17), (10, 21), (11, 12), (12, 14), (13, 20), (14, 17), (15, 17), (16, 19), (17, 16), (18, 5), (19, 0), (20, 0), (21, 0), (22, 0), (23, 0)]


### Do Zoology exhibitions get better ratings than other types?

In [79]:
cur.execute("""WITH ZoologyAvgRating AS (
                SELECT AVG(rating_id) AS zoology_avg_rating
                FROM review
                JOIN exhibition USING(exhibition_id)
                JOIN department USING(department_id)
                WHERE department_name = 'Zoology'
            ),
            OtherDepartmentsAvgRating AS (
                SELECT AVG(rating_id) AS other_departments_avg_rating
                FROM review
                JOIN exhibition USING(exhibition_id)
                JOIN department USING(department_id)
                WHERE department_name <> 'Zoology'
            )
            SELECT 
                CASE 
                    WHEN zoology_avg_rating > other_departments_avg_rating THEN 'Zoology exhibitions have better ratings'
                    WHEN zoology_avg_rating < other_departments_avg_rating THEN 'Zoology exhibitions have worse ratings'
                    ELSE 'Zoology exhibitions have similar ratings to other departments'
                END AS rating_comparison
            FROM ZoologyAvgRating, OtherDepartmentsAvgRating;""")

print(cur.fetchall())

[('Zoology exhibitions have better ratings',)]


### What exhibition has the most emergencies?

In [80]:
cur.execute("""SELECT exhibition_name, COUNT(*) AS emergency_count
            FROM exhibition
            JOIN request USING(exhibition_id)
            JOIN assistance a USING(assistance_id)
            WHERE a.description = 'emergency'
            GROUP BY exhibition_name
            ORDER BY emergency_count DESC""")

print(cur.fetchone())

('The Crenshaw Collection', 2)


### Which exhibitions receive fewer assistance requests than the average?

In [81]:
cur.execute("""WITH AvgAssistanceRequests AS (
                SELECT AVG(request_count) AS avg_request_count
                FROM (
                    SELECT COUNT(*) AS request_count
                    FROM request
                    GROUP BY exhibition_id
                ) AS exhibition_requests
            )
            SELECT exhibition_name
            FROM exhibition
            JOIN (
                SELECT exhibition_id, COUNT(*) AS request_count
                FROM request
                GROUP BY exhibition_id
            ) AS exhibition_requests USING(exhibition_id)
            CROSS JOIN AvgAssistanceRequests
            WHERE exhibition_requests.request_count < avg_request_count;""")

print(cur.fetchall())

[('Measureless to Man',), ('Adaptation',), ('Our Polluted World',), ('The Crenshaw Collection',)]


### Are there particular times when assistance requests/emergencies are more likely?

In [82]:
cur.execute("""WITH AssistanceRequestsByHour AS (
                SELECT EXTRACT(HOUR FROM created_at) AS hour_of_day,
                    COUNT(*) AS total_requests,
                    SUM(CASE WHEN assistance.description = 'emergency' THEN 1 ELSE 0 END) AS emergency_requests
                FROM request
                JOIN assistance USING(assistance_id)
                GROUP BY hour_of_day
            )
            SELECT hour_of_day::INT,
                total_requests,
                emergency_requests,
                emergency_requests::FLOAT / total_requests AS emergency_ratio
            FROM AssistanceRequestsByHour
            ORDER BY hour_of_day;""")

print(cur.fetchall())

[(8, 2, 0, 0.0), (9, 6, 0, 0.0), (10, 8, 0, 0.0), (11, 7, 1, 0.14285714285714285), (12, 7, 0, 0.0), (13, 4, 0, 0.0), (14, 11, 0, 0.0), (15, 6, 0, 0.0), (16, 9, 0, 0.0), (17, 8, 1, 0.125), (18, 5, 0, 0.0)]


### Are emergencies more likely in exhibitions that are more exciting than others?

In [83]:
cur.execute("""WITH ExhibitionAvgRating AS (
                SELECT exhibition_id, AVG(rating_id) AS avg_rating
                FROM exhibition
                JOIN review USING(exhibition_id)
                GROUP BY exhibition_id
            ),
            ExhibitionEmergencyCount AS (
                SELECT exhibition_id, COUNT(*) AS emergency_count
                FROM exhibition
                JOIN request USING(exhibition_id)
                JOIN assistance a USING(assistance_id)
                WHERE a.description = 'emergency'
                GROUP BY exhibition_id
            )
            SELECT 
                CASE 
                    WHEN emergency_count > avg_rating THEN 'Exhibitions with more emergencies than average rating'
                    WHEN emergency_count < avg_rating THEN 'Exhibitions with fewer emergencies than average rating'
                    ELSE 'Exhibitions with similar emergencies and average rating'
                END AS emergency_comparison
            FROM ExhibitionEmergencyCount
            JOIN ExhibitionAvgRating USING(exhibition_id);""")

print(cur.fetchone())

('Exhibitions with more emergencies than average rating',)


### Which floors are above average for ratings?

In [84]:
cur.execute("""WITH AvgFloorRating AS (
                SELECT floor_id, AVG(rating_id) AS avg_rating
                FROM exhibition
                JOIN review USING(exhibition_id)
                GROUP BY floor_id
            ),
            OverallAvgRating AS (
                SELECT AVG(rating_id) AS overall_avg_rating
                FROM review
            )
            SELECT floor_name
            FROM AvgFloorRating
            JOIN floor USING(floor_id)
            CROSS JOIN OverallAvgRating
            WHERE avg_rating > overall_avg_rating;""")

print(cur.fetchall())

[('Vault',), ('1',)]
